# Expanded Simulation Scenarios for EV Fleet Charging Optimization

This notebook implements the expanded simulation framework to address reviewer feedback.

## Simulation Plan Overview

### Phase 1: Scenario Diversity Analysis
- **S1 (Baseline)**: 140 EVs, 27% priority, normal conditions
- **S2 (Low Priority)**: 140 EVs, 15% priority
- **S3 (High Priority)**: 140 EVs, 40% priority
- **S4 (Morning Rush)**: Concentrated arrival pattern (7-9 AM)
- **S5 (Weather Impact)**: Increased energy demand (+20%)
- **S6 (Peak Stress)**: Combined high priority + weather impact

### Phase 2: Scalability Testing
Fleet sizes: 30, 60, 90, 120, 150, 180 EVs

### Phase 3: Sensitivity Analysis
- Priority ratio variations
- Energy demand variations
- Arrival pattern variations

### Phase 4: Fairness Quantification
- Jain's Fairness Index
- Waiting time distribution
- Service quality metrics

### Phase 5: Renewable Integration Analysis
- PV utilization patterns
- BESS effectiveness
- Grid interaction analysis

In [1]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import json

# Set plotting style
plt.style.use('seaborn-v0_8-paper')
sns.set_palette('husl')

print('Libraries imported successfully')

Libraries imported successfully


## 1. Session Data Generation Classes

In [2]:
class EVSession:
    """Represents a single EV charging session"""
    
    def __init__(self, session_id, arrival_time, departure_time, 
                 energy_demand, is_priority, initial_soc=20):
        self.session_id = session_id
        self.arrival_time = arrival_time
        self.departure_time = departure_time
        self.energy_demand = energy_demand  # kWh
        self.is_priority = is_priority
        self.initial_soc = initial_soc  # %
        
    def to_dict(self):
        return {
            'SessionID': self.session_id,
            'ArrivalTime': self.arrival_time.strftime('%H:%M'),
            'DepartureTime': self.departure_time.strftime('%H:%M'),
            'EnergyDemand_kWh': self.energy_demand,
            'IsPriority': 1 if self.is_priority else 0,
            'InitialSOC_%': self.initial_soc
        }

class ScenarioGenerator:
    """Generates EV charging scenarios with different characteristics"""
    
    def __init__(self, seed=42):
        np.random.seed(seed)
        
    def generate_baseline_scenario(self, n_evs=140, priority_ratio=0.27):
        """S1: Baseline scenario with normal conditions"""
        sessions = []
        base_date = datetime(2024, 1, 1)
        
        for i in range(n_evs):
            # Normal arrival distribution (8 AM - 6 PM peak)
            arrival_hour = np.random.choice(
                range(6, 20), 
                p=self._normal_arrival_distribution()
            )
            arrival_min = np.random.randint(0, 60)
            arrival = base_date.replace(hour=arrival_hour, minute=arrival_min)
            
            # Parking duration: 2-8 hours
            duration_hours = np.random.uniform(2, 8)
            departure = arrival + timedelta(hours=duration_hours)
            
            # Energy demand: 15-45 kWh (mean ~30 kWh)
            energy_demand = np.random.uniform(15, 45)
            
            # Priority assignment
            is_priority = np.random.random() < priority_ratio
            
            # Initial SOC: 15-35%
            initial_soc = np.random.uniform(15, 35)
            
            session = EVSession(i+1, arrival, departure, energy_demand, 
                              is_priority, initial_soc)
            sessions.append(session)
            
        return sessions
    
    def generate_morning_rush_scenario(self, n_evs=140, priority_ratio=0.27):
        """S4: Morning rush with concentrated arrivals (7-9 AM)"""
        sessions = []
        base_date = datetime(2024, 1, 1)
        
        # 60% of EVs arrive between 7-9 AM
        n_rush = int(n_evs * 0.6)
        
        for i in range(n_evs):
            if i < n_rush:
                # Morning rush arrivals
                arrival_hour = np.random.choice([7, 8])
                arrival_min = np.random.randint(0, 60)
            else:
                # Distributed throughout the day
                arrival_hour = np.random.choice(range(6, 20))
                arrival_min = np.random.randint(0, 60)
                
            arrival = base_date.replace(hour=arrival_hour, minute=arrival_min)
            duration_hours = np.random.uniform(2, 8)
            departure = arrival + timedelta(hours=duration_hours)
            energy_demand = np.random.uniform(15, 45)
            is_priority = np.random.random() < priority_ratio
            initial_soc = np.random.uniform(15, 35)
            
            session = EVSession(i+1, arrival, departure, energy_demand, 
                              is_priority, initial_soc)
            sessions.append(session)
            
        return sessions
    
    def generate_weather_impact_scenario(self, n_evs=140, priority_ratio=0.27):
        """S5: Weather impact with increased energy demand (+20%)"""
        sessions = self.generate_baseline_scenario(n_evs, priority_ratio)
        
        # Increase energy demand by 20% to simulate cold weather
        for session in sessions:
            session.energy_demand *= 1.2
            # Also slightly lower initial SOC
            session.initial_soc = max(10, session.initial_soc - 5)
            
        return sessions
    
    def _normal_arrival_distribution(self):
        """Returns probability distribution for normal arrival pattern"""
        # Hours from 6 AM to 7 PM (14 hours)
        # Peak at 8-9 AM and 5-6 PM
        probs = np.array([0.03, 0.05, 0.15, 0.12, 0.08, 0.08, 0.08, 
                         0.08, 0.08, 0.08, 0.05, 0.05, 0.04, 0.03])
        return probs / probs.sum()

# Test the generator
gen = ScenarioGenerator(seed=42)
test_sessions = gen.generate_baseline_scenario(n_evs=10)
print(f'Generated {len(test_sessions)} test sessions')
print('Sample session:', test_sessions[0].to_dict())

Generated 10 test sessions
Sample session: {'SessionID': 1, 'ArrivalTime': '10:28', 'DepartureTime': '13:34', 'EnergyDemand_kWh': 38.39073000818308, 'IsPriority': 0, 'InitialSOC_%': 23.91665505707182}


## 2. Generate All Scenarios (S1-S6)

In [3]:
# Initialize generator
generator = ScenarioGenerator(seed=42)

# Generate all scenarios
scenarios = {}

# S1: Baseline
scenarios['S1_Baseline'] = generator.generate_baseline_scenario(
    n_evs=45, priority_ratio=0.27
)

# S2: Low Priority
scenarios['S2_LowPriority'] = generator.generate_baseline_scenario(
    n_evs=45, priority_ratio=0.15
)

# S3: High Priority  
scenarios['S3_HighPriority'] = generator.generate_baseline_scenario(
    n_evs=45, priority_ratio=0.40
)

# S4: Morning Rush
scenarios['S4_MorningRush'] = generator.generate_morning_rush_scenario(
    n_evs=45, priority_ratio=0.27
)

# S5: Weather Impact
scenarios['S5_WeatherImpact'] = generator.generate_weather_impact_scenario(
    n_evs=45, priority_ratio=0.27
)

# S6: Peak Stress (High Priority + Weather)
scenarios['S6_PeakStress'] = generator.generate_weather_impact_scenario(
    n_evs=45, priority_ratio=0.40
)

# Print summary
for name, sessions in scenarios.items():
    n_priority = sum(1 for s in sessions if s.is_priority)
    avg_demand = np.mean([s.energy_demand for s in sessions])
    print(f'{name}: {len(sessions)} EVs, {n_priority} priority ({n_priority/len(sessions)*100:.1f}%), '
          f'Avg demand: {avg_demand:.1f} kWh')

S1_Baseline: 45 EVs, 11 priority (24.4%), Avg demand: 31.2 kWh
S2_LowPriority: 45 EVs, 6 priority (13.3%), Avg demand: 30.2 kWh
S3_HighPriority: 45 EVs, 18 priority (40.0%), Avg demand: 28.4 kWh
S4_MorningRush: 45 EVs, 12 priority (26.7%), Avg demand: 30.5 kWh
S5_WeatherImpact: 45 EVs, 19 priority (42.2%), Avg demand: 38.3 kWh
S6_PeakStress: 45 EVs, 22 priority (48.9%), Avg demand: 39.6 kWh


## 3. Save Scenarios to Excel

In [ ]:
# Create Excel file with all scenarios
output_file = 'EV_Fleet_Scenarios_S1_S6.xlsx'

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    for name, sessions in scenarios.items():
        # Convert to DataFrame
        df = pd.DataFrame([s.to_dict() for s in sessions])
        
        # Write to sheet
        df.to_excel(writer, sheet_name=name, index=False)
        
print(f'Scenarios saved to {output_file}')
print(f'Total sheets: {len(scenarios)}')

## 4. Scenario Visualization and Statistics

In [ ]:
# Create comparison visualizations
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Scenario Comparison: Arrival Patterns', fontsize=16, fontweight='bold')

for idx, (name, sessions) in enumerate(scenarios.items()):
    ax = axes[idx // 3, idx % 3]
    
    # Extract arrival hours
    arrival_hours = [s.arrival_time.hour for s in sessions]
    
    # Plot histogram
    ax.hist(arrival_hours, bins=range(6, 21), alpha=0.7, edgecolor='black')
    ax.set_xlabel('Hour of Day')
    ax.set_ylabel('Number of Arrivals')
    ax.set_title(name)
    ax.grid(True, alpha=0.3)
    
plt.tight_layout()
plt.savefig('Scenario_Arrival_Patterns.png', dpi=300, bbox_inches='tight')
print('Arrival pattern visualization saved')

In [ ]:
# Energy demand comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Box plot of energy demands
data_for_box = []
labels_for_box = []
for name, sessions in scenarios.items():
    demands = [s.energy_demand for s in sessions]
    data_for_box.append(demands)
    labels_for_box.append(name.replace('_', '\n'))

axes[0].boxplot(data_for_box, labels=labels_for_box)
axes[0].set_ylabel('Energy Demand (kWh)')
axes[0].set_title('Energy Demand Distribution Across Scenarios')
axes[0].grid(True, alpha=0.3)
axes[0].tick_params(axis='x', rotation=45)

# Priority ratio comparison
priority_ratios = []
scenario_names = []
for name, sessions in scenarios.items():
    n_priority = sum(1 for s in sessions if s.is_priority)
    ratio = n_priority / len(sessions) * 100
    priority_ratios.append(ratio)
    scenario_names.append(name.replace('_', '\n'))

axes[1].bar(range(len(scenario_names)), priority_ratios, alpha=0.7, edgecolor='black')
axes[1].set_xticks(range(len(scenario_names)))
axes[1].set_xticklabels(scenario_names, rotation=45, ha='right')
axes[1].set_ylabel('Priority Vehicle Ratio (%)')
axes[1].set_title('Priority Vehicle Distribution')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('Scenario_Characteristics.png', dpi=300, bbox_inches='tight')
print('Scenario characteristics visualization saved')

## 5. Generate Scalability Test Scenarios

In [ ]:
# Generate scenarios for different fleet sizes
fleet_sizes = [30, 60, 90, 120, 150, 180]
scalability_scenarios = {}

for size in fleet_sizes:
    scenario_name = f'Scale_{size}EVs'
    scalability_scenarios[scenario_name] = generator.generate_baseline_scenario(
        n_evs=size, priority_ratio=0.27
    )
    
print('Scalability scenarios generated:')
for name, sessions in scalability_scenarios.items():
    print(f'  {name}: {len(sessions)} EVs')

In [ ]:
# Save scalability scenarios
output_file_scale = 'EV_Fleet_Scalability_Scenarios.xlsx'

with pd.ExcelWriter(output_file_scale, engine='openpyxl') as writer:
    for name, sessions in scalability_scenarios.items():
        df = pd.DataFrame([s.to_dict() for s in sessions])
        df.to_excel(writer, sheet_name=name, index=False)
        
print(f'Scalability scenarios saved to {output_file_scale}')

## 6. Summary Statistics Table

In [ ]:
# Create summary table
summary_data = []

for name, sessions in scenarios.items():
    n_total = len(sessions)
    n_priority = sum(1 for s in sessions if s.is_priority)
    priority_ratio = n_priority / n_total * 100
    
    demands = [s.energy_demand for s in sessions]
    avg_demand = np.mean(demands)
    std_demand = np.std(demands)
    total_demand = np.sum(demands)
    
    durations = [(s.departure_time - s.arrival_time).total_seconds() / 3600 
                 for s in sessions]
    avg_duration = np.mean(durations)
    
    summary_data.append({
        'Scenario': name,
        'Total EVs': n_total,
        'Priority EVs': n_priority,
        'Priority Ratio (%)': f'{priority_ratio:.1f}',
        'Avg Demand (kWh)': f'{avg_demand:.1f}',
        'Std Demand (kWh)': f'{std_demand:.1f}',
        'Total Demand (kWh)': f'{total_demand:.1f}',
        'Avg Duration (h)': f'{avg_duration:.1f}'
    })

summary_df = pd.DataFrame(summary_data)
print('\nScenario Summary Statistics:')
print(summary_df.to_string(index=False))

# Save to Excel
summary_df.to_excel('Scenario_Summary_Statistics.xlsx', index=False)
print('\nSummary statistics saved to Scenario_Summary_Statistics.xlsx')

## 7. Next Steps

After generating these scenarios, you should:

1. Run your AQPC algorithm on each scenario (S1-S6)
2. Run baseline algorithms (RR, LLF) for comparison
3. Collect performance metrics:
   - Priority fulfillment rate
   - Total charging cost
   - Non-priority vehicle fairness (Jain's Index)
   - Computation time
   - PV/BESS utilization

4. Generate comparison tables and figures for the paper

### Expected Results Structure:
- Table: Performance comparison across S1-S6
- Figure: Priority fulfillment rates by scenario
- Figure: Cost vs. priority guarantee trade-off
- Figure: Fairness metrics comparison
- Figure: Scalability analysis (computation time vs. fleet size)